## Inspect and processing data manually

In [ ]:
%%sh
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', sep='\t', 
                   compression='gzip', error_bad_lines=False, dtype='str')
data.dropna(inplace=True)

b'Skipping line 85458: expected 15 fields, saw 22\nSkipping line 91161: expected 15 fields, saw 22\n'
b'Skipping line 166123: expected 15 fields, saw 22\n'
b'Skipping line 225458: expected 15 fields, saw 22\nSkipping line 229936: expected 15 fields, saw 22\nSkipping line 259297: expected 15 fields, saw 22\n'
b'Skipping line 284728: expected 15 fields, saw 22\nSkipping line 286334: expected 15 fields, saw 22\nSkipping line 293400: expected 15 fields, saw 22\nSkipping line 294415: expected 15 fields, saw 22\nSkipping line 308150: expected 15 fields, saw 22\nSkipping line 315022: expected 15 fields, saw 22\nSkipping line 315730: expected 15 fields, saw 22\nSkipping line 316071: expected 15 fields, saw 22\nSkipping line 326729: expected 15 fields, saw 22\n'
b'Skipping line 329101: expected 15 fields, saw 22\nSkipping line 333077: expected 15 fields, saw 22\nSkipping line 377031: expected 15 fields, saw 22\nSkipping line 389496: expected 15 fields, saw 22\nSkipping line 390486: expected 15 

In [3]:
print(data.shape)
print(data.columns)

(1800755, 15)
Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')


In [4]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,2975964,R1NBG94582SJE2,B00I01JQJM,860486164,GoPro Rechargeable Battery 2.0 (HERO3/HERO3+ o...,Camera,5,0,0,N,Y,Five Stars,ok,2015-08-31
1,US,23526356,R273DCA6Y0H9V7,B00TCO0ZAA,292641483,Professional 58mm Center Pinch Lens Cap for CA...,Camera,5,0,0,N,Y,Love it!!!,"Perfect, even sturdier than the original!",2015-08-31
2,US,52764145,RQVOXO7WUOFK6,B00B7733E0,75825744,Spy Tec Z12 Motion Activated Intelligent Secur...,Camera,2,1,1,N,Y,Another Motion Detect Fail,"If the words, &#34;Cheap Chinese Junk&#34; com...",2015-08-31
3,US,47348933,R1KWKSF21PO6HO,B006ZN4U34,789352955,"Celestron UpClose G2 10x25 Monocular, Black (7...",Camera,5,0,0,N,Y,Exactly what I wanted and expected.,Exactly what I wanted and expected. Perfect fo...,2015-08-31
4,US,33680700,R38H3UO1J190GI,B00HUEBGMU,19067902,Vidpro XM-L Wired Lavalier microphone - 20' Au...,Camera,5,1,1,N,Y,Good mic at a Good Price...Not Canon Though.,I will look past the fact that they tricked me...,2015-08-31


In [5]:
data = data[:100000]

In [6]:
data = data[['star_rating', 'review_body']]

In [7]:
data.star_rating.unique()

array(['5', '2', '3', '1', '4'], dtype=object)

In [8]:
data['label'] = data.star_rating.map({
    '1': '__label__negative__',
    '2': '__label__negative__',
    '3': '__label__neutral__',
    '4': '__label__positive__',
    '5': '__label__positive__'})

In [9]:
data = data.drop(['star_rating'], axis=1)

In [10]:
data = data[['label', 'review_body']]

In [11]:
data.head()

,label,review_body
0,__label__positive__,ok
1,__label__positive__,"Perfect, even sturdier than the original!"
2,__label__negative__,"If the words, &#34;Cheap Chinese Junk&#34; com..."
3,__label__positive__,Exactly what I wanted and expected. Perfect fo...
4,__label__positive__,I will look past the fact that they tricked me...


In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
%%time
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

CPU times: user 49.7 s, sys: 254 ms, total: 50 s
Wall time: 50 s


In [14]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)

CPU times: user 2.89 s, sys: 35.9 ms, total: 2.92 s
Wall time: 2.92 s


In [15]:
data.head()

,label,review_body
0,__label__positive__,ok
1,__label__positive__,"perfect , even sturdier than the original !"
2,__label__negative__,"if the words , & # 34 ; cheap chinese junk & #..."
3,__label__positive__,exactly what i wanted and expected . perfect f...
4,__label__positive__,i will look past the fact that they tricked me...


In [16]:
from sklearn.model_selection import train_test_split

training, validation = train_test_split(data, test_size=0.05)

In [17]:
print(training.shape)
print(validation.shape)

(95000, 2)
(5000, 2)


In [18]:
np.savetxt('/tmp/training.txt', training.values, fmt='%s')
np.savetxt('/tmp/validation.txt', validation.values, fmt='%s')

In [19]:
!head -5 /tmp/training.txt

__label__positive__ good
__label__positive__ great product
__label__positive__ these cables work perfectly , have done so for the past two years for me , and are a great value .
__label__positive__ nice case . room for extra battery , card
__label__negative__ not a bad little device , when it worked . this thing is really fragile . it can take no , i repeat no abuse at all . i routinely would place it on my lapel every day before going out and about , just with basic daily use the face plate fell off , i was like no problem , a bit of super glue and good as new . a month later it refused to turn on . now it 's in the garbage . if you want toss away spec ops gear , something for one special use , then this could work , but you pay the risk of it failing on duty with whatever application you may want it to . if you need a small camera for some sort of critical task , or for daily use this is a very poor choice . but hey , at this price , do n't listen to me , get it and watch it fail in 

## Training

In [20]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

2.19.0


In [21]:
# Run this cell if you want to use the data you processed manually

prefix = 'amazon-reviews'

s3_train_path = session.upload_data(path='/tmp/training.txt', bucket=bucket, key_prefix=prefix+'/input/train')
s3_val_path = session.upload_data(path='/tmp/validation.txt', bucket=bucket, key_prefix=prefix+'/input/validation')
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

s3://sagemaker-eu-west-1-613904931467/amazon-reviews/input/train/training.txt
s3://sagemaker-eu-west-1-613904931467/amazon-reviews/input/validation/validation.txt
s3://sagemaker-eu-west-1-613904931467/amazon-reviews/output/


In [ ]:
# Run this cell if you want to use the data processed by SageMaker Processing

prefix = 'amazon-reviews'

s3_train_path = 
s3_val_path = 
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [22]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('blazingtext', region)
print(container)

685385470294.dkr.ecr.eu-west-1.amazonaws.com/blazingtext:1


In [23]:
role = sagemaker.get_execution_role()

bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.c5.2xlarge',
                                   output_path=s3_output)

In [24]:
bt.set_hyperparameters(mode='supervised')

In [25]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                      distribution='FullyReplicated', 
                      content_type='text/plain',
                      s3_data_type='S3Prefix')

validation_data = sagemaker.TrainingInput(s3_val_path,
                      distribution='FullyReplicated', 
                      content_type='text/plain', 
                      s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

In [26]:
bt.fit(inputs=s3_channels)

2020-12-17 09:19:34 Starting - Starting the training job...
2020-12-17 09:19:56 Starting - Launching requested ML instancesProfilerReport-1608196773: InProgress
......
2020-12-17 09:20:57 Starting - Preparing the instances for training...
2020-12-17 09:21:29 Downloading - Downloading input data
2020-12-17 09:21:29 Training - Downloading the training image..Arguments: train
[12/17/2020 09:21:44 WARNING 140429841462912] Loggers have already been setup.
[12/17/2020 09:21:44 WARNING 140429841462912] Loggers have already been setup.
[12/17/2020 09:21:45 INFO 140429841462912] nvidia-smi took: 0.025170326232910156 secs to identify 0 gpus
[12/17/2020 09:21:45 INFO 140429841462912] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[12/17/2020 09:21:45 INFO 140429841462912] Processing /opt/ml/input/data/train/training.txt . File size: 25.360501289367676 MB
[12/17/2020 09:21:45 INFO 140429841462912] Processing /opt/ml/input/data/v

In [27]:
bt_predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-----------------!

In [28]:
import json
import pprint

sentences = ['This is a bad camera it doesnt work at all , i want a refund  .', 
             'The camera works , the pictures are decent quality, nothing special to say about it .',
             'Very happy to have bought this , exactly what I needed']

payload = {"instances" : sentences, "configuration": {"k": 3}}

bt_predictor.content_type = 'application/json'
response = bt_predictor.predict(json.dumps(payload))

print(response)

AttributeError: can't set attribute

In [ ]:
bt_predictor.delete_endpoint()